# 데이터 확인

In [3]:
import pandas as pd 

df = pd.read_csv('./results/2nd-dataset_서울 송파구_20250530_141219.csv')

# 데이터의 상위 5행을 출력하여 확인
print("--- 원본 데이터 상위 5행 ---")
print(df.head())

# 데이터의 전체적인 정보 (컬럼별 데이터 타입, 결측치 여부) 확인
print("\n--- 데이터 정보 ---")
df.info()

# 각 컬럼별 고유한 값의 개수 확인 (특히 범주형 데이터)
print("\n--- 고유 값 개수 확인 ---")
for col in df.columns:
    if df[col].dtype == 'object' or col in ['has_playoff', 'is_holiday']: # object 타입이거나 0, 1로 이루어진 컬럼
        print(f"{col}: {df[col].nunique()}개") # nunique()는 고유한 값의 개수를 세어줍니다.

--- 원본 데이터 상위 5행 ---
         date region_code  accident_count  game_count sport_types  \
0  2023-01-01      서울 송파구               5           1          농구   
1  2023-01-02      서울 송파구               5           0          없음   
2  2023-01-03      서울 송파구               4           0          없음   
3  2023-01-04      서울 송파구              10           1          농구   
4  2023-01-05      서울 송파구               8           1          농구   

   has_playoff  temperature  precipitation  snow_depth weather_condition  \
0            0         -2.0            0.0         0.0                맑음   
1            0         -5.2            0.0         0.0                맑음   
2            0         -6.2            0.0         0.0                맑음   
3            0         -3.5            0.0         0.0                맑음   
4            0         -3.6            0.0         0.0              구름조금   

   is_holiday weekday  
0           1       일  
1           0       월  
2           0       화  
3          

# 결측치 확인

In [4]:
# 결측치 확인 (NaN으로 표시된 부분이 있는지)
print("\n--- 컬럼별 결측치 개수 ---")
print(df.isnull().sum())

# 만약 'temperature' 컬럼에 결측치가 있었다면, 평균으로 대체
if df['temperature'].isnull().sum() > 0:
    df['temperature'].fillna(df['temperature'].mean(), inplace=True)
    print("\n'temperature' 컬럼 결측치를 평균으로 대체 완료.")

# 만약 'snow_depth' 컬럼에 결측치가 있었다면, 최빈값으로 대체
if df['snow_depth'].isnull().sum() > 0:
    df['snow_depth'].fillna(df['snow_depth'].mode()[0], inplace=True)
    print("\n'snow_depth' 컬럼 결측치를 최빈값으로 대체 완료.")


--- 컬럼별 결측치 개수 ---
date                 0
region_code          0
accident_count       0
game_count           0
sport_types          0
has_playoff          0
temperature          0
precipitation        0
snow_depth           0
weather_condition    0
is_holiday           0
weekday              0
dtype: int64


# 날짜 시간 데이터 확인

In [5]:
# 'date' 컬럼을 datetime 객체로 변환
df['date'] = pd.to_datetime(df['date'])

# 연, 월, 일, 요일, 주차 정보 추출
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek # 월요일:0, 화요일:1, ..., 일요일:6
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int) # 연중 몇 번째 주인지

# 'date' 컬럼은 더 이상 필요 없으므로 삭제
df.drop('date', axis=1, inplace=True)

print("\n--- 날짜 데이터 처리 후 데이터 상위 5행 ---")
print(df.head())
print("\n--- 날짜 데이터 처리 후 데이터 정보 ---")
df.info()


--- 날짜 데이터 처리 후 데이터 상위 5행 ---
  region_code  accident_count  game_count sport_types  has_playoff  \
0      서울 송파구               5           1          농구            0   
1      서울 송파구               5           0          없음            0   
2      서울 송파구               4           0          없음            0   
3      서울 송파구              10           1          농구            0   
4      서울 송파구               8           1          농구            0   

   temperature  precipitation  snow_depth weather_condition  is_holiday  \
0         -2.0            0.0         0.0                맑음           1   
1         -5.2            0.0         0.0                맑음           0   
2         -6.2            0.0         0.0                맑음           0   
3         -3.5            0.0         0.0                맑음           0   
4         -3.6            0.0         0.0              구름조금           0   

  weekday  year  month  day  day_of_week  week_of_year  
0       일  2023      1    1            6

# 범주형 데이터 인코딩

In [6]:
# 'region_code' 원-핫 인코딩
# pd.get_dummies는 원-핫 인코딩을 쉽게 해줍니다.
# drop_first=True는 첫 번째 범주를 삭제하여 다중공선성(multicollinearity) 문제를 피하는 데 도움을 줍니다.
df = pd.get_dummies(df, columns=['region_code'], prefix='region', drop_first=True)

# 'sport_types' 다중 레이블 원-핫 인코딩
# 먼저 쉼표로 구분된 스포츠 종류를 분리합니다.
df['sport_types'] = df['sport_types'].apply(lambda x: x.split(',') if pd.notnull(x) else [])

# 각 스포츠 타입에 대한 새로운 컬럼 생성
all_sport_types = set()
for types in df['sport_types']:
    for s_type in types:
        all_sport_types.add(s_type.strip()) # strip()으로 공백 제거

for s_type in all_sport_types:
    # "없음"은 스포츠가 아니므로 제외
    if s_type != '없음':
        df[f'sport_{s_type}'] = df['sport_types'].apply(lambda x: 1 if s_type in x else 0)

# 원본 'sport_types' 컬럼은 삭제
df.drop('sport_types', axis=1, inplace=True)

# 'weather_condition' 원-핫 인코딩
# drop_first=True로 해서 하나의 컬럼은 삭제 (모든 컬럼이 False이면 구름조금)
df = pd.get_dummies(df, columns=['weather_condition'], prefix='weather', drop_first=True)

# 'weekday' 컬럼은 'day_of_week'와 중복되므로 삭제
df.drop('weekday', axis=1, inplace=True)

print("\n--- 범주형 데이터 인코딩 후 데이터 상위 5행 ---")
print(df.head())
print("\n--- 범주형 데이터 인코딩 후 데이터 정보 ---")
df.info()


--- 범주형 데이터 인코딩 후 데이터 상위 5행 ---
   accident_count  game_count  has_playoff  temperature  precipitation  \
0               5           1            0         -2.0            0.0   
1               5           0            0         -5.2            0.0   
2               4           0            0         -6.2            0.0   
3              10           1            0         -3.5            0.0   
4               8           1            0         -3.6            0.0   

   snow_depth  is_holiday  year  month  day  day_of_week  week_of_year  \
0         0.0           1  2023      1    1            6            52   
1         0.0           0  2023      1    2            0             1   
2         0.0           0  2023      1    3            1             1   
3         0.0           0  2023      1    4            2             1   
4         0.0           0  2023      1    5            3             1   

   sport_야구  sport_농구  weather_맑음  weather_비/눈  weather_흐림  
0         0     

# 피처 스케일링

In [7]:
from sklearn.preprocessing import StandardScaler

# 스케일링 대상 컬럼 지정
# 'has_playoff', 'is_holiday', 'region_서울 송파구', 'sport_*', 'weather_*'는 0 또는 1이므로 스케일링할 필요 없음.
numeric_cols = [
    'accident_count', 'game_count', 'temperature', 'precipitation', 'snow_depth'
]

# StandardScaler 객체 생성
scaler = StandardScaler()

# 스케일링 대상 컬럼에 표준화 적용
# fit_transform은 데이터를 학습(fit)하고 변환(transform)하는 것을 한 번에 수행합니다.
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print("\n--- 피처 스케일링 후 데이터 상위 5행 ---")
print(df.head())
print("\n--- 피처 스케일링 후 데이터 정보 ---")
df.info()


--- 피처 스케일링 후 데이터 상위 5행 ---
   accident_count  game_count  has_playoff  temperature  precipitation  \
0       -0.688841    0.444049            0    -1.523787      -0.300351   
1       -0.688841   -1.031404            0    -1.833155      -0.300351   
2       -1.033970   -1.031404            0    -1.929832      -0.300351   
3        1.036802    0.444049            0    -1.668803      -0.300351   
4        0.346545    0.444049            0    -1.678471      -0.300351   

   snow_depth  is_holiday  year  month  day  day_of_week  week_of_year  \
0   -0.107904           1  2023      1    1            6            52   
1   -0.107904           0  2023      1    2            0             1   
2   -0.107904           0  2023      1    3            1             1   
3   -0.107904           0  2023      1    4            2             1   
4   -0.107904           0  2023      1    5            3             1   

   sport_야구  sport_농구  weather_맑음  weather_비/눈  weather_흐림  
0         0         

# 최종데이터

In [8]:
print("\n--- 최종 전처리 완료 데이터 상위 5행 ---")
print(df.head())
print("\n--- 최종 전처리 완료 데이터 정보 ---")
df.info()

df


--- 최종 전처리 완료 데이터 상위 5행 ---
   accident_count  game_count  has_playoff  temperature  precipitation  \
0       -0.688841    0.444049            0    -1.523787      -0.300351   
1       -0.688841   -1.031404            0    -1.833155      -0.300351   
2       -1.033970   -1.031404            0    -1.929832      -0.300351   
3        1.036802    0.444049            0    -1.668803      -0.300351   
4        0.346545    0.444049            0    -1.678471      -0.300351   

   snow_depth  is_holiday  year  month  day  day_of_week  week_of_year  \
0   -0.107904           1  2023      1    1            6            52   
1   -0.107904           0  2023      1    2            0             1   
2   -0.107904           0  2023      1    3            1             1   
3   -0.107904           0  2023      1    4            2             1   
4   -0.107904           0  2023      1    5            3             1   

   sport_야구  sport_농구  weather_맑음  weather_비/눈  weather_흐림  
0         0         

,accident_count,game_count,has_playoff,temperature,precipitation,snow_depth,is_holiday,year,month,day,day_of_week,week_of_year,sport_야구,sport_농구,weather_맑음,weather_비/눈,weather_흐림
0,-0.688841,0.444049,0,-1.523787,-0.300351,-0.107904,1,2023,1,1,6,52,0,1,True,False,False
1,-0.688841,-1.031404,0,-1.833155,-0.300351,-0.107904,0,2023,1,2,0,1,0,0,True,False,False
2,-1.033970,-1.031404,0,-1.929832,-0.300351,-0.107904,0,2023,1,3,1,1,0,0,True,False,False
3,1.036802,0.444049,0,-1.668803,-0.300351,-0.107904,0,2023,1,4,2,1,0,1,True,False,False
4,0.346545,0.444049,0,-1.678471,-0.300351,-0.107904,0,2023,1,5,3,1,0,1,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,-1.033970,-1.031404,0,-1.591461,-0.300351,-0.093312,0,2024,12,27,4,52,0,0,True,False,False
727,-0.343712,-1.031404,0,-1.630132,-0.300351,-0.086015,1,2024,12,28,5,52,0,0,True,False,False
728,-0.688841,-1.031404,0,-1.282094,-0.300351,-0.107904,1,2024,12,29,6,52,0,0,False,False,False
729,-1.033970,-1.031404,0,-0.847045,-0.300351,-0.107904,0,2024,12,30,0,1,0,0,False,False,False


# 피처 엔지니어링

In [9]:
# 체감 온도 근사치: 기온 - (강수량 * 상수)
# 강수량이 많을수록 체감 온도가 낮아진다고 가정
df['perceived_temperature'] = df['temperature'] - (df['precipitation'] * 0.1)

print("\n--- 'perceived_temperature' 피처 추가 후 데이터 상위 5행 ---")
print(df[['temperature', 'precipitation', 'perceived_temperature']].head())


--- 'perceived_temperature' 피처 추가 후 데이터 상위 5행 ---
   temperature  precipitation  perceived_temperature
0    -1.523787      -0.300351              -1.493752
1    -1.833155      -0.300351              -1.803120
2    -1.929832      -0.300351              -1.899797
3    -1.668803      -0.300351              -1.638768
4    -1.678471      -0.300351              -1.648436


In [10]:
# 'weather_비/눈' 컬럼이 1이거나 snow_depth가 0보다 클 경우 악천후로 간주
# df['weather_비/눈'] 컬럼은 원-핫 인코딩 후에 생성되었을 것입니다.
df['is_bad_weather'] = ((df['weather_비/눈'] == 1) | (df['snow_depth'] > 0)).astype(int)

print("\n--- 'is_bad_weather' 피처 추가 후 데이터 상위 5행 ---")
print(df[['weather_비/눈', 'snow_depth', 'is_bad_weather']].head())


--- 'is_bad_weather' 피처 추가 후 데이터 상위 5행 ---
   weather_비/눈  snow_depth  is_bad_weather
0        False   -0.107904               0
1        False   -0.107904               0
2        False   -0.107904               0
3        False   -0.107904               0
4        False   -0.107904               0


In [11]:
# 플레이오프 경기에는 가중치 2를 부여한다고 가정 (임의의 가중치)
df['total_game_activity'] = df['game_count'] + (df['has_playoff'] * df['game_count'] * 1.0) # has_playoff가 1이면 게임 수 2배

print("\n--- 'total_game_activity' 피처 추가 후 데이터 상위 5행 ---")
print(df[['game_count', 'has_playoff', 'total_game_activity']].head())


--- 'total_game_activity' 피처 추가 후 데이터 상위 5행 ---
   game_count  has_playoff  total_game_activity
0    0.444049            0             0.444049
1   -1.031404            0            -1.031404
2   -1.031404            0            -1.031404
3    0.444049            0             0.444049
4    0.444049            0             0.444049


In [12]:
# 온도와 강수량의 곱
df['temp_precip_interaction'] = df['temperature'] * df['precipitation']

# 온도가 낮으면서 강수량이 많은 경우를 강조 (예: -기온 * 강수량)
# 온도가 낮을수록 양수값이 커지도록 (가정)
df['cold_wet_interaction'] = -df['temperature'] * df['precipitation']

print("\n--- 'temp_precip_interaction', 'cold_wet_interaction' 피처 추가 후 데이터 상위 5행 ---")
print(df[['temperature', 'precipitation', 'temp_precip_interaction', 'cold_wet_interaction']].head())


--- 'temp_precip_interaction', 'cold_wet_interaction' 피처 추가 후 데이터 상위 5행 ---
   temperature  precipitation  temp_precip_interaction  cold_wet_interaction
0    -1.523787      -0.300351                 0.457672             -0.457672
1    -1.833155      -0.300351                 0.550591             -0.550591
2    -1.929832      -0.300351                 0.579628             -0.579628
3    -1.668803      -0.300351                 0.501227             -0.501227
4    -1.678471      -0.300351                 0.504131             -0.504131


In [13]:
# 'accident_count'가 목표 변수이므로, 이 피처는 모델 훈련 시 주의해서 사용해야 합니다.
# 주로 시계열 예측 모델에서 과거 목표 변수를 피처로 사용합니다.
# 여기서는 피처 엔지니어링의 예시로만 보여드립니다.

# 날짜 순서로 정렬 (필수)
df_sorted = df.sort_values(by=['year', 'month', 'day'])

# 최근 3일 평균 사고 수 (rolling window)
# df_sorted['accident_count'].shift(1)는 이전 날짜의 사고 수를 가져옵니다.
# .rolling(window=3)은 3일(3개 행)의 윈도우를 만듭니다.
# .mean()은 그 윈도우 내의 평균을 계산합니다.
# min_periods=1은 윈도우 크기가 3이 안 되더라도 계산하도록 합니다. (처음 2일간은 1일, 2일 데이터로만 평균)
df['past_3day_avg_accident'] = df_sorted['accident_count'].shift(1).rolling(window=3, min_periods=1).mean()
# 결측치(NaN, 데이터 초반부에 이전 데이터가 없어서 생김)는 0으로 채우거나 다른 방식으로 처리할 수 있습니다.
# 여기서는 단순화를 위해 NaN 그대로 둡니다.

print("\n--- 'past_3day_avg_accident' 피처 추가 후 데이터 상위 5행 ---")
print(df[['day', 'accident_count', 'past_3day_avg_accident']].head())


--- 'past_3day_avg_accident' 피처 추가 후 데이터 상위 5행 ---
   day  accident_count  past_3day_avg_accident
0    1       -0.688841                     NaN
1    2       -0.688841               -0.688841
2    3       -1.033970               -0.688841
3    4        1.036802               -0.803884
4    5        0.346545               -0.228669


In [14]:
print("\n--- 최종 피처 엔지니어링 완료 데이터 상위 5행 ---")
print(df.head())
print("\n--- 최종 피처 엔지니어링 완료 데이터 정보 ---")
df.info()
df


--- 최종 피처 엔지니어링 완료 데이터 상위 5행 ---
   accident_count  game_count  has_playoff  temperature  precipitation  \
0       -0.688841    0.444049            0    -1.523787      -0.300351   
1       -0.688841   -1.031404            0    -1.833155      -0.300351   
2       -1.033970   -1.031404            0    -1.929832      -0.300351   
3        1.036802    0.444049            0    -1.668803      -0.300351   
4        0.346545    0.444049            0    -1.678471      -0.300351   

   snow_depth  is_holiday  year  month  day  ...  sport_농구  weather_맑음  \
0   -0.107904           1  2023      1    1  ...         1        True   
1   -0.107904           0  2023      1    2  ...         0        True   
2   -0.107904           0  2023      1    3  ...         0        True   
3   -0.107904           0  2023      1    4  ...         1        True   
4   -0.107904           0  2023      1    5  ...         1       False   

   weather_비/눈  weather_흐림  perceived_temperature  is_bad_weather  \
0      

,accident_count,game_count,has_playoff,temperature,precipitation,snow_depth,is_holiday,year,month,day,...,sport_농구,weather_맑음,weather_비/눈,weather_흐림,perceived_temperature,is_bad_weather,total_game_activity,temp_precip_interaction,cold_wet_interaction,past_3day_avg_accident
0,-0.688841,0.444049,0,-1.523787,-0.300351,-0.107904,1,2023,1,1,...,1,True,False,False,-1.493752,0,0.444049,0.457672,-0.457672,NaN
1,-0.688841,-1.031404,0,-1.833155,-0.300351,-0.107904,0,2023,1,2,...,0,True,False,False,-1.803120,0,-1.031404,0.550591,-0.550591,-0.688841
2,-1.033970,-1.031404,0,-1.929832,-0.300351,-0.107904,0,2023,1,3,...,0,True,False,False,-1.899797,0,-1.031404,0.579628,-0.579628,-0.688841
3,1.036802,0.444049,0,-1.668803,-0.300351,-0.107904,0,2023,1,4,...,1,True,False,False,-1.638768,0,0.444049,0.501227,-0.501227,-0.803884
4,0.346545,0.444049,0,-1.678471,-0.300351,-0.107904,0,2023,1,5,...,1,False,False,False,-1.648436,0,0.444049,0.504131,-0.504131,-0.228669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,-1.033970,-1.031404,0,-1.591461,-0.300351,-0.093312,0,2024,12,27,...,0,True,False,False,-1.561426,0,-1.031404,0.477998,-0.477998,-0.113626
727,-0.343712,-1.031404,0,-1.630132,-0.300351,-0.086015,1,2024,12,28,...,0,True,False,False,-1.600097,0,-1.031404,0.489613,-0.489613,-0.918927
728,-0.688841,-1.031404,0,-1.282094,-0.300351,-0.107904,1,2024,12,29,...,0,False,False,False,-1.252058,0,-1.031404,0.385079,-0.385079,-0.573798
729,-1.033970,-1.031404,0,-0.847045,-0.300351,-0.107904,0,2024,12,30,...,0,False,False,False,-0.817010,0,-1.031404,0.254411,-0.254411,-0.688841
